In [1]:
from __future__ import print_function
import os
import re
import logging
import SimpleITK as sitk
import numpy as np
import pandas as pd
import time
import itertools
from scipy.spatial.distance import pdist, squareform
from scipy.optimize import curve_fit
from scipy.integrate import quad, trapz


In [2]:
local_directory = 'Radiomics features/'
oar_names = ['spleen', 'kidney_right', 'kidney_left', 'gallbladder', 
             'liver', 'stomach', 'pancreas', 'prostate' ]
oar_indices = [1, 2, 3, 4, 
              5, 6, 7, 8]
node_feature_length = 107 # needs to be same for all images!
doses = ['D1', 'D2', 'D3', 'D4', 'D5', 'D6']
timepoints = [24, 48, 72, 96, 120, 144, 168]
ranges = {
    'range1': [24, 48, 72],
    'range2': [96, 120],
    'range3': [144, 168]
}
external_directory = '/Volumes/My Passport/RPT/Patientwise_data_original'
local_directory = 'Radiomics features'

Clinical bloodworks data

In [3]:
excel_file = '../bloodwork.xlsx'
bloodwork = pd.ExcelFile(excel_file)

In [4]:
def calculate_tia_3tp(extended_activities, extended_timepoints):
    (c1, k1), _ = curve_fit(monoexponential, extended_timepoints[:3], extended_activities[:3], 
                            p0=[extended_activities[0], 0.1], bounds=(0, np.inf))
    (c2, k2), _ = curve_fit(monoexponential, extended_timepoints[-3:], extended_activities[-3:],
                            p0=[extended_activities[-3], 0.1], bounds=(0, np.inf))

    c2 = monoexponential(timepoints[2], c1, k1)/np.exp(-k2*timepoints[2]) 

    # Calculate the TIA as the sum of the integrals of the two fitted curves
    integral_1, _ = quad(monoexponential, 0, extended_timepoints[2], 
                         args=(c1, k1))
    integral_2, _ = quad(monoexponential, extended_timepoints[2], 
                         extended_timepoints[-1], args=(c2, k2))
    tia_3tp = integral_1 + integral_2
    return tia_3tp/1e6

def calculate_tia_3tp_alternate(extended_activities, extended_timepoints):
    trapezoidal_area = trapz(extended_activities[:4], extended_timepoints[:4])
    (c2, k2), _ = curve_fit(monoexponential, extended_timepoints[2:4], extended_activities[2:4],
                            p0=[extended_activities[2], 0.1], bounds=(0, np.inf))
    c2_adjusted = extended_activities[3] / np.exp(-k2 * extended_timepoints[3])
    monoexponential_area, _ = quad(lambda t: monoexponential(t, c2_adjusted, k2), 
                                   extended_timepoints[3], np.inf)

    # The total TIA is the sum of the trapezoidal area and the monoexponential area
    tia = trapezoidal_area + monoexponential_area
    return tia/1e6

def tia_madsen_calculation(a, t):
    # Ensure k_pop is negative since it's a decay constant
    k_pop = -0.004
    t = t
    tia = a * (np.exp(k_pop * t) / -k_pop)
    return tia/1e6

def tia_hanscheid_calculation(a, t):
    tia = a * (2 * t / np.log(2))
    return tia/1e6

In [132]:
def monoexponential(t, c, k):
    return c * np.exp(-k * t)

def fit_3tp_tia(subject_folder, dose, timepoints):
    oar_wise_activity = {oar:[] for oar in oar_indices}
    extended_timepoints = [0] + timepoints + [600]
    fit_3tp_tia = pd.DataFrame(columns = ['OAR', 'TIA'])
    for timepoint in timepoints:
        spect_filename = os.path.join(external_directory, 
                            f"{subject_folder}/{subject_folder}_{dose}_{timepoint}_SPECT_reg.nii")
        seg_filename = os.path.join(external_directory, 
                            f"{subject_folder}/{subject_folder}_{dose}_{timepoint}_CT_seg.nii")
        
        spect_image = sitk.ReadImage(spect_filename)
        seg_image = sitk.ReadImage(seg_filename)
        
        spect_array = sitk.GetArrayFromImage(spect_image)
        seg_array = sitk.GetArrayFromImage(seg_image)
        
        for oar_index in oar_indices:
            oar_mask = seg_array == oar_index
            mean_oar_activity = spect_array[oar_mask].sum() if np.any(oar_mask) else 0
            oar_wise_activity[oar_index].append(mean_oar_activity)
    
    fit_3tp_tia = pd.DataFrame(columns = ['OAR'])
    
    for oar_index, activities in oar_wise_activity.items():
        if not np.any(np.isnan(activities)):
            extended_activities = [0] + activities + [600]
            tia_3tp = calculate_tia_3tp_alternate(extended_activities, extended_timepoints)
            
            tia_madsen_values = [tia_madsen_calculation(act, tp) for act, tp in zip(activities, timepoints)]
            tia_hanscheid_values = [tia_hanscheid_calculation(act, tp) for act, tp in zip(activities, timepoints)]

            # Append all results to the DataFrame for this OAR
            fit_3tp_tia = fit_3tp_tia.append({
                'OAR': oar_names[oar_index - 1],  # Adjust index as necessary
                'TIA_3TP': tia_3tp,
            }, ignore_index=True)
            
        
    return fit_3tp_tia, tia_madsen_values, tia_hanscheid_values

In [133]:
fit_3tp_tia('P00007', 'D4', [24, 120, 168])

/Users/vibhabalaji/anaconda3/envs/PSMA_SPECT/lib/python3.6/site-packages/scipy/optimize/minpack.py:829: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
/Users/vibhabalaji/anaconda3/envs/PSMA_SPECT/lib/python3.6/site-packages/ipykernel_launcher.py:23: IntegrationWarning: The integral is probably divergent, or slowly convergent.


(            OAR       TIA_3TP
 0        spleen   1927.963371
 1  kidney_right  12157.875288
 2   kidney_left  12107.699515
 3   gallbladder    140.314096
 4         liver  22165.729386
 5       stomach    492.528480
 6      pancreas    680.014948
 7      prostate    184.495795,
 [131.4788174215676, 154.2285117948736, 432.8562046341766],
 [40.08887113636158, 345.20079820090507, 1643.4759253867894])

In [17]:
spect_file_pattern = re.compile(rf"{subject_id}_D(\d+)_(\d+)_SPECT.nii")
range1_features = {}
range2_features = {}
range3_features = {}

for subject_folder in os.listdir(local_directory):
    subject_path = os.path.join(local_directory, subject_folder)
    if not os.path.isdir(subject_path):
        continue
    print(f"Subject {subject_folder}------")
    
    for dose in doses:
        available_timepoints = []
        
        for file in os.listdir(subject_path):
            match_spect = re.match(rf"{subject_folder}_{dose}_(\d+)_radiomic_features.pkl", file)
            if match:
                tp = int(match.group(1))
                available_timepoints.append(tp)
                
        if len(available_timepoints) == 3:
            print(f"Dose {dose} has 3 timepoints: {available_timepoints}")
            gt_3tp_tia, tia_madsen, tia_handcheid = fit_3tp_tia(subject_folder, dose, timepoints)
            for timepoint in available_timepoints:
                file_node_features = node_features(subject_folder, dose, timepoint)
                file_edge_features = edge_features(subject_folder, dose, timepoint)
                if timepoint in [24, 48, 72]:
                    range1_features.append((file_node_features, file_edge_features, gt_3tp_tia))
                elif timepoint in [96, 120]:
                    range2_features.append((file_node_features, file_edge_features, gt_3tp_tia))
                else:
                    range3_features.append((file_node_features, file_edge_features, gt_3tp_tia))

In [109]:
def node_features(subject_folder, dose, timepoint):
    file = os.path.join(local_directory, 
                f'{subject_folder}/{subject_folder}_{dose}_{timepoint}_radiomic_features.pkl')
    subject_features = pd.read_pickle(file)
    subject_node_features = {}
    for oar_index in oar_indices:
        if oar_index in subject_features.columns:
            subject_node_features[oar_index] = subject_features[oar_index].values.flatten()
        else:
            subject_node_features[oar_index] = np.zeros(node_feature_length)
    return subject_node_features

In [108]:
def calculate_distances(mask1, mask2):
    indices1 = np.argwhere(mask1)
    indices2 = np.argwhere(mask2)
    distances = pdist(np.vstack((indices1, indices2)), metric = 'euclidean')
    num_points1 = len(indices1)
    num_points2 = len(indices2)
    inter_distances = distances[num_points1*(num_points1-1)//2 + num_points2*(num_points2-1)//2:]
    return inter_distances

def edge_features(subject_folder, dose, timepoint):
    seg = os.path.join(external_directory, 
                       f'{subject_folder}/{subject_folder}_{dose}_{timepoint}_seg.nii')
    seg_array = sitk.GetArayFromImage(sitk.ReadImage(ct_seg_image))
    oar_labels = np.unique(seg_array)[1:]
    adjacency_matrix = np.zeros((8,8))
    edge_features = {}
    for oar1 in oar_indices:
        for oar2 in oar_indices:
            if i < j:
                mask1 = seg_array == oar1
                mask2 = seg_array == oar2

                inter_distances = calculate_distances(mask1, mask2)

                if inter_distances.size>0:
                    min_dist = np.min(inter_distances)
                    max_dist = np.max(inter_distances)
                    mean_dist = np.mean(inter_distances)
                    median_dist = np.median(inter_distances)
                    
                    adjacency_matrix[oar1-1, oar2-1] = 1
                    adjacency_matrix[oar2-1, oar1-1] = 1
                    edge_features[(oar1, oar2)] = {
                        'min_dist': min_dist,
                        'max_dist': max_dist,
                        'mean_dist': mean_dist,
                        'median_dist': median_dist
                    }
                else:
                    adjacency_matrix[oar1-1, oar2-1] = 1
                    adjacency_matrix[oar2-1, oar1-1] = 1

    return adjacency_matrix, edge_features